In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import datetime
#turns seconds into a string format ffmpeg uses
def seconds_formatter(sec):
    x = "0" + str(datetime.timedelta(seconds=sec))[:-3]
    return x

## Random Forest Classifier
identify reaches, successes, failures

### Training Data

# Project Variables 
### Change as needed.

In [33]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\A_copiedDeepCut_resnet50_goprolensfrontJun25shuffle1_60002.h5")
#remove extra level "scorer"
df.columns = df.columns.droplevel()

#Change the video length!!!
vid_length = 459

total_num_frames = df.shape[0]

cols = ['Nose', 'DomInside', 'Pellet', 'Index']

classes = ['n', 's', 'f', 'r']

frames_before = 16

frames_after = 32

step_size = 1

seed = 74

In [14]:
def frame_to_time(frame):
    total_seconds = (frame/total_num_frames)*vid_length
    return round(total_seconds, 3)
def time_to_frame(time):
    return int(round(total_num_frames*(time/vid_length)))

### Functions for Creating Classifier

In [15]:
import random
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

#Functions to get training data
def get_dataset(start_frames):
    dataframe = df[cols]
    dataset = []
    for start_f in start_frames:
        temp_df = dataframe.iloc[start_f - frames_before: start_f + frames_after]
        dataset.append(temp_df.values.flatten())
    return np.asarray(dataset)
def get_predicted_frames(classifier):
    dataframe = df[cols]
    inputs = []
    num_frames = frames_before + frames_after
    for index in range(num_frames, dataframe.shape[0] - num_frames, step_size):
        temp_df = dataframe.iloc[index - frames_before: index + frames_after]
        inputs.append(temp_df.values.flatten())
    inputs = np.asarray(inputs)
    predictions = classifier.predict(inputs)
    return predictions
def get_non_reach_frames(num_frames, blacklist, avoidance_radius):
    non_reaches = []
    random.seed(seed)
    while len(non_reaches) < num_frames:
        #hard code 3 non-reaches
        frame = random.randint(frames_before, total_num_frames - frames_after)

        too_close = False
        for start_time in blacklist:
            if abs(frame - start_time) < avoidance_radius:
                print('no')
                too_close = True
                break
        if not too_close:
            non_reaches.append(frame)
    return non_reaches

#Use training features and labels to create Random Forest Classifier
def create_classifier(features, labels, n_trees, test_proportion=0.25):
    train_feats, test_feats, train_labs, test_labs = train_test_split(features, labels, test_size = test_proportion, random_state = seed)
    print('Training Features Shape:', train_feats.shape)
    print('Training Labels Shape:', train_labs.shape)
    print('Testing Features Shape:', test_feats.shape)
    print('Testing Labels Shape:', test_labs.shape)
    rf = RandomForestClassifier(n_estimators = n_trees, random_state = seed)
    # Train the model on training data
    print(train_feats.shape)
    print(train_labs.shape)
    rf.fit(train_feats, train_labs)
    # Use the forest's predict method on the test data
    predictions = rf.predict(test_feats)
    print(predictions)
    print(test_labs)
    wrong_labels = []
    for i in range(len(predictions)):
        if predictions[i] != test_labs[i]:
            frame = -1
            for key in frame_to_features.keys():
                if (frame_to_features[key] == test_feats[i]).all():
                    frame = key
                    break
            wrong_labels.append((frame_to_time(frame), predictions[i], test_labs[i]))
            predictions[i] = 'X'
    print('Wrong Labels', wrong_labels)
    return rf, wrong_labels

In [16]:
true_reach_times = [1.153, 1.844, 14.02, 14.538, 15.724, 16.227, 18.416, 21.184, 27.747, 32.237, 33.952, 37.731, 
                38.378, 41.412, 42.066, 42.364, 45.69, 48.501, 50.928, 55.111, 56.685, 61.739, 62.336, 
                62.747, 64.524, 66.032, 76.965, 83.258, 90.252, 105.25, 105.67, 109.465, 113.356, 113.994, 121.798,
                124.059, 132.204, 132.597, 135.263, 135.633, 136.588, 138.327, 145.304, 154.300, 170.547, 
                182.046, 190.137, 202.976, 240.157, 243.168, 253.75, 258.548, 272.461, 281.791, 284.774, 
                295.074, 303.63, 310.3, 323.374, 351.034, 364.535, 376.356, 432.654]
true_reach_frames = [time_to_frame(x) for x in true_reach_times]
true_reach_labels = ['r', 'r', 'f', 'f', 'r', 'r', 'r', 's', 'r', 'r', 's', 'r', 'r', 'r', 'f', 'r', 'f', 
                     'r', 's', 'r', 's', 'r', 'r', 'r', 'r', 's', 's', 'f', 'f', 'f', 'r', 'r', 'f', 'f',
                     'r', 's', 'f', 'r', 'r', 'r', 'r', 's', 's', 's', 's', 'f', 's', 's', 'r', 's', 'f', 
                     'f', 's', 'r', 's', 's', 'r', 's', 'f', 'f', 'f', 's', 's']
non_reaches_frames = get_non_reach_frames(70, true_reach_frames, 12)
features = get_dataset(true_reach_frames+non_reaches_frames)
labels = np.asarray(true_reach_labels + ['n' for x in non_reaches_frames])
frame_to_features = dict(zip(true_reach_frames+non_reaches_frames, features))

no
no
no


In [17]:
blue = create_classifier(features, labels, 1000, test_proportion=0.01)

Training Features Shape: (131, 576)
Training Labels Shape: (131,)
Testing Features Shape: (2, 576)
Testing Labels Shape: (2,)
(131, 576)
(131,)
['n' 's']
['n' 's']
Wrong Labels []


In [12]:
wrong_ones = []
for i in range(100, 200):
    seed = i
    wrong_ones.extend(create_classifier(features, labels, 1000, test_proportion=0.1)[1])
wrong_ones.sort(key= lambda x: x[0])
wrong_df = pd.DataFrame(wrong_ones, columns = ['time', 'guess', 'actual'])
wrong_df.groupby(wrong_df.columns.tolist(),as_index=False).size()

Training Features Shape: (119, 1728)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1728)
Testing Labels Shape: (14,)
(119, 1728)
(119,)
['f' 'n' 's' 'n' 'f' 'n' 's' 'n' 's' 'r' 'n' 'f' 'f' 'n']
['f' 'n' 's' 'n' 'f' 'n' 's' 'n' 's' 'r' 'n' 'f' 'f' 'n']
Wrong Labels []
Training Features Shape: (119, 1728)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1728)
Testing Labels Shape: (14,)
(119, 1728)
(119,)
['s' 'n' 'n' 'f' 'n' 'r' 'n' 'n' 'n' 'n' 's' 'r' 'r' 'r']
['s' 'n' 'n' 'f' 'n' 'r' 'n' 'n' 'n' 'n' 's' 'r' 'r' 'r']
Wrong Labels []
Training Features Shape: (119, 1728)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1728)
Testing Labels Shape: (14,)
(119, 1728)
(119,)
['f' 's' 'n' 'r' 'r' 'n' 's' 'n' 'n' 'n' 'r' 'n' 'n' 's']
['f' 's' 'n' 'r' 'r' 'n' 's' 'n' 'n' 'n' 'f' 'n' 'n' 's']
Wrong Labels [(351.036, 'r', 'f')]
Training Features Shape: (119, 1728)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1728)
Testing Labels Shape: (14,)
(119, 17

['r' 'n' 'f' 'n' 'r' 'r' 'r' 'n' 'n' 'n' 'f' 'n' 'n' 'f']
['r' 'n' 'f' 'n' 'r' 'r' 'r' 'n' 'n' 'n' 'r' 'n' 'n' 'f']
Wrong Labels [(240.16, 'f', 'r')]
Training Features Shape: (119, 1728)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1728)
Testing Labels Shape: (14,)
(119, 1728)
(119,)
['n' 's' 's' 'n' 'n' 'n' 'n' 'f' 'n' 'r' 'n' 'n' 'n' 'r']
['n' 's' 's' 'n' 'n' 'n' 'n' 'f' 'n' 'r' 'n' 'n' 'n' 'r']
Wrong Labels []
Training Features Shape: (119, 1728)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1728)
Testing Labels Shape: (14,)
(119, 1728)
(119,)
['r' 'n' 'n' 'n' 'r' 'n' 's' 'r' 'n' 'n' 'f' 'n' 'n' 's']
['r' 'n' 'n' 'n' 'r' 'n' 's' 'r' 'n' 'n' 'f' 'n' 'n' 's']
Wrong Labels []
Training Features Shape: (119, 1728)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1728)
Testing Labels Shape: (14,)
(119, 1728)
(119,)
['f' 'n' 'f' 's' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'r']
['f' 'n' 'f' 's' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'r']
Wrong Labels []
Trainin

KeyboardInterrupt: 

In [11]:
wrong_df
wrong_df.groupby(wrong_df.columns.tolist(),as_index=False).size()

NameError: name 'wrong_df' is not defined

In [109]:
# 182: pellet goes up like a success?
# 240: pellet dispenser goes down during reach
# 83, 90: mouse drops pellet within/near acrylic, ambigious even to humans
# 

# Find Success, Fails, and Reaches

In [38]:
import operator
#Helper Functions
def get_predicted_times(classifier, df):
    dataframe = df[cols]
    inputs = []
    num_frames = frames_before + frames_after
    for index in range(num_frames, dataframe.shape[0] - num_frames, step_size):
        temp_df = dataframe.iloc[index - frames_before: index + frames_after]
        inputs.append(temp_df.values.flatten())
    inputs = np.asarray(inputs)
    predictions = classifier.predict(inputs)
    return predictions, list(range(num_frames, dataframe.shape[0] - num_frames, step_size))
def get_starts(lst):
    starts = [lst[0]]
    for i in range(1, len(lst)):
        if lst[i] - lst[i-1] > 0.2:
            starts.append(lst[i])
    return starts

#Find Reaches, 
def get_classified_starts(classifier, df):
    predictions2, start_frames = get_predicted_times(classifier, df)
    class_to_predictions = {}
    for start, prediction in zip(start_frames, predictions2):
        for c in classes:
            if prediction == c:
                if c in class_to_predictions.keys():
                    class_to_predictions[c].append(frame_to_time(start))
                else:
                    class_to_predictions[c] = [frame_to_time(start)]
    for c in classes:
        hold = get_starts(class_to_predictions[c])
        class_to_predictions[c] = hold
    return class_to_predictions
starts = get_classified_starts(blue[0], df)
predicted_fails = starts['f']
predicted_reaches = starts['r']
predicted_successes = starts['s']
print(predicted_fails)
print(predicted_reaches)
print(predicted_successes)
x = (predicted_fails + predicted_reaches + predicted_successes)

[13.989, 14.506, 42.041, 45.645, 50.875, 65.99, 83.207, 90.188, 105.211, 113.303, 113.953, 132.163, 154.268, 181.995, 240.102, 253.699, 258.495, 323.3, 350.986, 364.474]
[1.118, 1.793, 14.489, 15.665, 16.174, 18.376, 27.652, 32.173, 37.687, 38.313, 41.357, 42.3, 48.447, 55.046, 61.619, 62.311, 64.463, 105.629, 109.382, 121.744, 132.547, 133.181, 135.199, 135.591, 136.526, 240.11, 281.735, 303.581]
[1.977, 21.137, 33.917, 50.908, 56.631, 65.998, 76.925, 123.997, 135.725, 138.286, 145.259, 154.276, 170.509, 190.086, 202.924, 240.569, 243.105, 272.409, 284.729, 295.039, 310.246, 351.094, 376.302, 432.582]


In [37]:
df.shape[0]/120

458.55

In [32]:
y = sorted(x)
print(y)
import numpy as np
np.savetxt("file_name.csv", y, delimiter=",", fmt='%s', header='times')

[1.126, 1.802, 1.977, 13.989, 14.489, 14.514, 15.665, 16.191, 18.393, 21.146, 27.652, 32.182, 33.933, 37.687, 38.313, 41.366, 42.041, 42.316, 45.645, 48.447, 50.875, 50.925, 55.054, 56.631, 61.635, 62.311, 62.636, 64.463, 65.99, 66.015, 76.925, 83.207, 90.188, 105.228, 105.629, 109.382, 113.311, 113.962, 121.744, 123.997, 132.18, 132.555, 133.181, 135.208, 135.608, 135.758, 136.534, 138.286, 145.267, 154.276, 170.517, 182.003, 190.086, 202.924, 240.11, 240.586, 243.113, 253.699, 258.503, 272.417, 281.751, 284.729, 295.039, 303.598, 310.254, 323.317, 350.994, 351.094, 364.482, 376.319, 432.599]


In [13]:
def frame_to_time(frame):
    total_seconds = (frame/9351)*77.6
    return round(total_seconds, 3)
def time_to_frame(time):
    return int(round(9351*(time/77.6)))

In [137]:
frame_to_time(6)

0.05

In [15]:
reaches = [0.353, 12.325, 16.568, 17.55, 18.912, 20.41, 22.1, 22.92, 24.699, 27.4, 29.754, 30.431, 31.39, 32.692, 37.731, 
           38.258, 42.86, 45.714, 56.45, 76.803, 77.86, 86.2, 86.75, 91.13, 91.649, 97.563, 98.1, 106.739, 107.292, 116.412, 
           121.49, 122.236, 124.775, 125.375, 134.324, 134.859, 142.37, 143.0, 150.0, 150.955, 159.435, 160.04, 165.02, 
           165.633, 174.018, 177.129, 179.23, 183.719, 185.755, 192.041, 197.214, 198.66, 201.286, 203.34, 204.15, 206.253, 
           209.36, 213.027, 213.895, 217.676, 218.325, 218.982, 223.198, 226.56, 230.524, 233.61, 234.286, 235.838, 236.316, 
           237.091, 250.448, 251.206, 253.05]
reaches_labels = ['r', 's', 'r', 'f', 'r', 'r', 'f', 'r', 'r', 'r', 'f', 'r', 'r', 'r?', 'f', 'r', 'r', 's', 'r', 'f', 'r', 
                  'f', 'r', 'f', 'r', 'f', 'r', 'f', 'r', 'f?', 'r', 'r', 'f', 'r', 'f', 'r', 'f', 'r', 'f', '?', 'f', 'r', 
                  'f', 'r', 's', 'r', 's', 'f', 'r', 's', 'f', 'r', 'f', 'r', 'r', 's', 'r', 'f', 'r', 'r', 'f', 'r', 's', 
                  'r', 's', 'r', 'r', 'f', 'f', 'f?', 'f', 'r', 'r']
s = [reaches[i] for i in range(len(reaches)) if reaches_labels[i] == 's']
r = [reaches[i] for i in range(len(reaches)) if reaches_labels[i] == 'r']
f = [reaches[i] for i in range(len(reaches)) if reaches_labels[i] == 'f']

In [16]:
df2 = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\6_26_2.7k_copyDeepCut_resnet50_goprolensfrontJun25shuffle1_60002.h5")
df2.columns = df2.columns.droplevel()
starts = get_classified_starts(blue[0], df2)
predicted_fails = starts['f']
predicted_reaches = starts['r']
predicted_successes = starts['s']
print(predicted_fails)
print(predicted_reaches)
print(predicted_successes)

[16.282]
[5.377, 5.826, 6.423, 11.402, 13.195, 13.643, 14.191, 16.182, 16.68, 17.477, 19.668, 20.315, 21.858, 23.9, 27.087, 31.468, 31.916, 33.012, 33.659, 35.601, 45.758, 50.986, 51.484, 54.82, 59.302, 59.75, 60.895, 62.14, 63.434, 65.775, 66.82, 67.368, 72.297, 72.895, 75.185]
[6.473, 27.584, 45.808, 66.073]


In [163]:
path_config_file = r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25"

In [164]:
'if not exist "{}" mkdir {}'.format(path_config_file[:-11] + "\\vid_clips", path_config_file[:-11] + "vid_clips")

'if not exist "C:\\Users\\vjj14\\Desktop\\DeepLabCut\\goprolensfront-vj\\vid_clips" mkdir C:\\Users\\vjj14\\Desktop\\DeepLabCut\\goprolensfront-vjvid_clips'

In [167]:
#2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label
reaches2 = [11.59, 17.76, 18.47, 20.81, 22.45, 25.49, 37.40, 41.88, 51.55, 57.35, 64.50, 78.54, 84.61, 
            96.27, 108.61, 117.01, 123.13, 130.13, 157.55, 164.34, 207.55]
vids2 = ['11.59.mp4', '17.76.mp4', '18.47.mp4', '20.81.mp4', '22.45.mp4', '25.49.mp4', '37.4.mp4', '41.88.mp4', '51.55.mp4', 
         '57.35.mp4', '64.5.mp4', '78.54.mp4', '84.61.mp4', '96.27.mp4', '108.61.mp4', '117.01.mp4', '123.13.mp4', 
         '130.13.mp4', '157.55.mp4', '164.34.mp4', '207.55.mp4']
#2019.07.08_CC42973_N2INJURED_V3_2.7k_120FPS_Label
reaches3 = [21.19, 66.49, 71.22, 85.34, 92.22, 99.69, 116.29, 127.70, 135.33, 142.42, 147.11, 
                   151.60, 157.72, 163.40, 172.79, 183.10, 204.05, 208.73, 216.32]
vids3 = ['21.19.mp4', '66.49.mp4', '71.22.mp4', '85.34.mp4', '92.22.mp4', '99.69.mp4', '116.29.mp4', '127.7.mp4', 
         '135.33.mp4', '142.42.mp4', '147.11.mp4', '151.6.mp4', '157.72.mp4', '163.4.mp4', '172.79.mp4', '183.1.mp4', 
         '204.05.mp4', '208.73.mp4', '216.32.mp4']

In [178]:
import datetime
import numpy as np
project_name = 'gopro3lens'
path_config_file= r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\config.yaml"
video = "2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label"
#turns seconds into a string format ffmpeg uses
def seconds_formatter(sec):
    x = "0" + str(datetime.timedelta(seconds=sec))[:-3]
    return x

#Open up new batch file
bat_file= path_config_file[:-11] + "videos\{}_create_videos.bat".format(project_name)
directory = path_config_file[:-11] + "videos\\" + video
script_writer= open(bat_file,"w")

script_writer.write('if not exist "{}" mkdir {} && '.format(directory, directory))

#write series of shell commands to create videos for each attempt, connected by '&&' for the batch file
vids = []
for time in reaches2:
    start_time = seconds_formatter(max(0, time - 1.25))
    cmd = "ffmpeg -i {0}.mp4 -ss {1} -t {2} -c copy {3}{4}.mp4".format(video, start_time, 2, directory + "\\", np.round(time, 3))
    vids.append(str(np.round(time, 3)) + ".mp4")
    script_writer.write(cmd + " && ")
script_writer.close()

#open and write a list of videos to concatenate (.txt file) for ffmpeg to use
list_file = path_config_file[:-11] + "videos\{}_video_list.txt".format(project_name)
list_writer = open(list_file, "w+")
for v in vids:
    list_writer.write('file ' + "\'" + directory +"\{}".format(v) + "\'\n")
list_writer.close()

#open batch file again and write command to produce a concatenated video, the final product
script_writer = open(bat_file, "a+")
script_writer.write("ffmpeg -f concat -safe 0 -i {0} -c copy output2.mp4".format(list_file))
script_writer.close()

In [181]:
cmd = "ffmpeg -i {0}.mp4 -ss {1} -t {2} -c copy {3}{4}.mp4".format(video, start_time, 2, directory + "\\", np.round(time, 3))
print(cmd)
print("ffmpeg -f concat -safe 0 -i {0} -c copy output2.mp4".format(list_file))
directory

ffmpeg -i 2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label.mp4 -ss 00:03:26.300 -t 2 -c copy C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label\207.55.mp4
ffmpeg -f concat -safe 0 -i C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\gopro3lens_video_list.txt -c copy output2.mp4


'C:\\Users\\vjj14\\Desktop\\DeepLabCut\\gopro3lens-vj-2019-07-08\\videos\\2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label'

In [31]:
path_config_file= r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\config.yaml"

path_config_file[:-11] + "2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label" + "\\" + "asdf.mp4"

'C:\\Users\\vjj14\\Desktop\\DeepLabCut\\gopro3lens-vj-2019-07-08\\2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label\\asdf.mp4'

In [180]:
print(vids)

['11.59.mp4', '17.76.mp4', '18.47.mp4', '20.81.mp4', '22.45.mp4', '25.49.mp4', '37.4.mp4', '41.88.mp4', '51.55.mp4', '57.35.mp4', '64.5.mp4', '78.54.mp4', '84.61.mp4', '96.27.mp4', '108.61.mp4', '117.01.mp4', '123.13.mp4', '130.13.mp4', '157.55.mp4', '164.34.mp4', '207.55.mp4']
